In [ ]:
import warnings
warnings.simplefilter('always', DeprecationWarning)

from osgeo import gdal
from osgeo import ogr
from osgeo import osr
import numpy as np
import math
# from osgeo import gdal_array
# from osgeo import gdalnumeric

import os
import contextlib
import logging

gdal.UseExceptions()

DEBUG=True
logging.basicConfig(level=logging.DEBUG if DEBUG else logging.ERROR)

def silent_remove(path):
    with contextlib.suppress(FileNotFoundError):
        os.remove(path)

# Lecture de l'ENC

In [ ]:
enc_ds = ogr.Open('data/ENC_ROOT/US4MA13M/US4MA13M.000')
assert enc_ds.GetDriver().GetDescription() == 'S57' # S57
m_covr_lyr = enc_ds.GetLayerByName('M_COVR')
soundg_lyr = enc_ds.GetLayerByName('SOUNDG')
depare_lyr = enc_ds.GetLayerByName('DEPARE')

covr_geo = next(f.GetGeometryRef().Clone() for f in m_covr_lyr if f.GetField('CATCOV') == 1)
covr_geo

## Écriture de l'emprise dans `data/covr.geojson`

In [ ]:
json_drv = ogr.GetDriverByName('GeoJSON')
covr_path = 'data/covr.geojson'
def create_covr():
    silent_remove(covr_path)
    covr_ds = json_drv.CreateDataSource(covr_path)
    covr_lyr = covr_ds.CreateLayer('covr')
    covr_f = ogr.Feature(covr_lyr.GetLayerDefn())
    covr_f.SetGeometry(covr_geo)
    covr_lyr.CreateFeature(covr_f)
    covr_ds.Release()
create_covr()
del create_covr

# Lecture de la Bathy

## Réduction à la zone de l'ENC

In [ ]:
bathy_ds = gdal.Warp("data/bathy.tif", 'data/navd_bath_30m',
                        options = gdal.WarpOptions(
                                            format='GTiff', 
                                            cutlineDSName=covr_path,
                                            cutlineLayer='covr',
                                            cropToCutline=True,
                                            creationOptions=['COMPRESS=LZW'],
                                            dstNodata=np.nan))
bathy_ds.FlushCache()

In [ ]:
assert bathy_ds.RasterCount == 1
bathy_band = bathy_ds.GetRasterBand(1)
bathy_np = bathy_band.ReadAsArray()
bathy_band.GetNoDataValue()

In [ ]:
bathy_np.shape # Northing, Easting

# Essais sur les transformations géographiques

In [ ]:
# b = gdal.Open('data/navd_bath_30m')
# b = gdal.Open('data/bathy.tif')
b = bathy_ds
geotransform = b.GetGeoTransform()
print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))
print("???        = ({}, {})".format(geotransform[2], geotransform[4]))

In [ ]:
# b = gdal.Open('data/navd_bath_30m')
b = gdal.Open('data/bathy.tif')
# b = bathy_ds
geotransform = b.GetGeoTransform()
print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))
print("???        = ({}, {})".format(geotransform[2], geotransform[4]))

In [ ]:
geotransform

# PixelAccessor
PixelAccessor et tests

In [ ]:
class PixelAccessor:

    def __init__(self, geotransformSource, spatialRef = None):
        
        # Dataset
        if isinstance(geotransformSource, gdal.Dataset):
            geotransform  = geotransformSource.GetGeoTransform()
            # Check if same project
            if (spatialRef is not None
                and spatialRef.ExportToWkt() == geotransformSource.GetSpatialRef().ExportToWkt()):
                spatialRef == None # same: do nothing during geotransformation
            else:
                geoIn = osr.CreateCoordinateTransformation(spatialRef, geotransformSource.GetSpatialRef())
                geoOut = osr.CreateCoordinateTransformation(geotransformSource.GetSpatialRef(), spatialRef)

        # Band: get original Dataset
        elif isinstance(geotransformSource, gdal.Band):
            return self.__init__(geotransformSource.GetDataset(), spatialRef)

        # Simple tuple
        elif isinstance(geotransformSource, tuple):
            geotransform = geotransformSource
            if spatialRef is None:
                raise ValueError('spatialRef must be None when using geotransform tuple')
        else:
            raise TypeError('must be gdal.DataSet, gdal.Band or tuple')

        # Access to local variable is faster than acces throw self
        assert geotransform[2] == 0
        assert geotransform[4] == 0
        def pixelToGeo(L, P): # small matrix, faster than numpy
            Xp = geotransform[0] + (P+.5)*geotransform[1] 
            Yp = geotransform[3] + (L+.5)*geotransform[5]
            if spatialRef:
                return geoOut.TransformPoint(Xp, Yp)
            else:
                return Xp, Yp

        def geoToPixel(Xp, Yp):
            if spatialRef:
                Xp, Yp, Zp = geoIn.TransformPoint(Xp, Yp)
            P = math.floor((Xp - geotransform[0]) / geotransform[1])
            L = math.floor((Yp - geotransform[3]) / geotransform[5])
            return L, P

        # Publish method Javascript style
        self.pixelToGeo = pixelToGeo
        self.geoToPixel = geoToPixel


In [ ]:
t = PixelAccessor(bathy_band)

In [ ]:
t.pixelToGeo(0,0), t.pixelToGeo(2000,0), t.pixelToGeo(0, 2000)

In [ ]:
t.geoToPixel(328133.590626, 4722220.06), t.geoToPixel(388133.590626, 4722220.06), t.geoToPixel(328133.590626, 4662220.06)

In [ ]:
bathy_np[t.geoToPixel(415283.0,4717460.3)] # -125,9262

In [ ]:
tw84 = PixelAccessor(bathy_band, depare_lyr.GetSpatialRef())

In [ ]:
tw84.pixelToGeo(0,0), tw84.pixelToGeo(2000,0), tw84.pixelToGeo(0, 2000)

In [ ]:
tw84.geoToPixel(-71.09592403816357, 42.63312848595494), tw84.geoToPixel(-71.07804409258947, 42.09312188970164), tw84.geoToPixel(-70.36441180339453, 42.64417944703203)

In [ ]:
bathy_np[tw84.geoToPixel(-70.551710,42.36582775)] # -88.0935

# Parcours d'isobathe

In [ ]:
for isobath in depare_lyr:
    print(isobath)